In [69]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black
from pyspark.sql import functions as F, Window
from manga_recsys.spark import get_spark

spark = get_spark()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [74]:
from IPython.display import display
from manga_recsys.models.manga import (
    generate_manga_tags_word2vec,
    generate_recommendations,
    explode_recommendations,
    map_names_to_recommendations,
)

manga_info = spark.read.parquet(
    "../data/processed/2022-12-17-metadata-listing/manga_info.parquet"
)

manga_tags = generate_manga_tags_word2vec(manga_info, vector_col="w2v")
display(manga_tags.head())

rec_df = generate_recommendations(
    manga_tags, "id", "w2v", k=10, metric="cosine", n_jobs=8
)
display(rec_df.head())

recs = map_names_to_recommendations(manga_info, explode_recommendations(spark, rec_df))
recs.show()

,id,tags,w2v
0,0001183c-2089-48e9-96b7-d48db5f1a611,"[Psychological, Drama, Supernatural]","[1.0564969778060913, -0.7721579074859619, 0.88..."
1,00016bf9-455f-44e5-ab27-55ac7f69aad2,"[Oneshot, Sci-Fi, Post-Apocalyptic, Tragedy]","[1.7515220642089844, 0.12437917292118073, 1.53..."
2,000245bf-670e-49c5-af47-1d674a43525c,"[Comedy, Video Games, Slice of Life, Music]","[0.6973272562026978, -0.05136081576347351, 0.2..."
3,00034238-6f5d-4f98-a51c-69afb21d920f,"[Oneshot, Boys' Love, Doujinshi]","[-0.5257415175437927, 0.28495192527770996, 0.6..."
4,0003d7e8-31da-463b-97e9-e87045f8f5ee,"[Oneshot, Romance, Doujinshi, Drama, Slice of ...","[0.249611958861351, -0.18225233256816864, 0.64..."


f:\code\mangadex\.venv\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


,id,tags,w2v,recommendations,distances
0,0001183c-2089-48e9-96b7-d48db5f1a611,"[Psychological, Drama, Supernatural]","[1.0564969778060913, -0.7721579074859619, 0.88...","[2ab9e47a-df90-456e-9ce9-c0930cf3c390, 07b5841...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01672708..."
1,00016bf9-455f-44e5-ab27-55ac7f69aad2,"[Oneshot, Sci-Fi, Post-Apocalyptic, Tragedy]","[1.7515220642089844, 0.12437917292118073, 1.53...","[00016bf9-455f-44e5-ab27-55ac7f69aad2, 1b815f8...","[1.1920927089903444e-07, 1.1920927089903444e-0..."
2,000245bf-670e-49c5-af47-1d674a43525c,"[Comedy, Video Games, Slice of Life, Music]","[0.6973272562026978, -0.05136081576347351, 0.2...","[686426b9-f1cc-4faf-a345-f5471f19d5ac, 000245b...","[0.0, 0.0, 0.02981483786027672, 0.035659059981..."
3,00034238-6f5d-4f98-a51c-69afb21d920f,"[Oneshot, Boys' Love, Doujinshi]","[-0.5257415175437927, 0.28495192527770996, 0.6...","[5bad3fe1-c741-4766-805b-555949510aa6, 49e3f0f...","[-1.192093050939036e-07, -1.192093050939036e-0..."
4,0003d7e8-31da-463b-97e9-e87045f8f5ee,"[Oneshot, Romance, Doujinshi, Drama, Slice of ...","[0.249611958861351, -0.18225233256816864, 0.64...","[e1676745-3733-40a2-86a6-a1b3661b2900, 0003d7e...","[1.1920927089903444e-07, 1.1920927089903444e-0..."


f:\code\mangadex\.venv\lib\site-packages\pyspark\sql\pandas\conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


+--------------------+-------+--------------------+--------------------+--------------------+
|                  id|   name|              rec_id|            rec_name|            distance|
+--------------------+-------+--------------------+--------------------+--------------------+
|0001183c-2089-48e...|  Eight|0001183c-2089-48e...|               Eight|                 0.0|
|0001183c-2089-48e...|  Eight|2ab9e47a-df90-456...|       Mind Assassin|                 0.0|
|0001183c-2089-48e...|  Eight|07b5841e-1076-4b7...|                null|                 0.0|
|0001183c-2089-48e...|  Eight|f0acfa39-dfaf-40a...|                null|                 0.0|
|0001183c-2089-48e...|  Eight|65b6f5e2-dd4c-449...|                 HAL|                 0.0|
|0001183c-2089-48e...|  Eight|a457201c-8088-481...|       Space Ship EE|                 0.0|
|0001183c-2089-48e...|  Eight|c7e4a12e-837b-4c0...|     GhostApocalypse|                 0.0|
|0001183c-2089-48e...|  Eight|f656f7e1-78fb-4ff...|Kimi no K